In [0]:
%sql
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.delta;

In [0]:
events = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv", header=True, inferSchema=True)

#Converting csv to delta
events.write.format("delta").mode("overwrite").save("/Volumes/workspace/ecommerce/delta/events")

In [0]:
# Creating managed table
events.write.format("delta").saveAsTable("events_table")

In [0]:
%sql
DESCRIBE events_table

col_name,data_type,comment
event_time,timestamp,null
event_type,string,null
product_id,int,null
category_id,bigint,null
category_code,string,null
brand,string,null
price,double,null
user_id,int,null
user_session,string,null


In [0]:
#Trying to inserting invalid data-checking schema enforcement
bad_df = spark.createDataFrame(
    [(1, "not-a-timestamp", "click")],
    ["id", "event_time", "event_type"]
)

bad_df.write.format("delta").mode("append").saveAsTable("events_table")


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6079936943583534>, line 7
      1 #Trying to inserting invalid data-checking schema enforcement
      2 bad_df = spark.createDataFrame(
      3     [(1, "not-a-timestamp", "click")],
      4     ["id", "event_time", "event_type"]
      5 )
----> 7 bad_df.write.format("delta").mode("append").saveAsTable("events_table")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:737, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
    735 self._write.table_name = name
    736 self._write.table_save_method = "save_as_table"
--> 737 _, _, ei = self._spark.client.execute_command(
    738     self._write.command(self._spark.client), self._write.observations
    739 )
    740 self._callback(ei)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/cl

In [0]:
%sql
CREATE TABLE workspace.default.new_events (
  event_time TIMESTAMP,
  event_type STRING,
  product_id INT,
  category_id BIGINT,
  category_code STRING,
  brand STRING,
  price DOUBLE,
  user_id INT,
  user_session STRING
);

In [0]:
%sql
MERGE INTO events_table t
USING new_events s
ON t.user_id = s.user_id
WHEN MATCHED THEN
  UPDATE SET 
    t.price = s.price,
    t.brand = s.brand

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
0,0,0,0
